IMPORTS

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os
from pydicom import dcmread # for dcm files
import pydicom
import cv2
import csv

from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py


import tensorflow as tf
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# VISUALIZE SINGLE DCM FILE

def visualize_single_dcm_file(path)

In [ ]:

def visualize_single_dcm_file(path):
    
    dataset = dcmread(path)
    print("SOP name.....:", dataset.SOPClassUID)
    print("Series name.....:", dataset.SeriesInstanceUID)

    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)


    print("Image position...:", dataset.ImagePositionPatient )
    cut_constant = 50
    pixel_arrays_shape = (dataset.pixel_array).shape
    crop = dataset.pixel_array[:,cut_constant:pixel_arrays_shape[1]-1-cut_constant]
    crop = crop[cut_constant:pixel_arrays_shape[0]-1-cut_constant,:]
    image = np.asarray(dataset.pixel_array)

    print(image.shape)
    #print(crop.shape)
    inter_area_resize_256 = cv2.resize(dataset.pixel_array , (256, 256),  interpolation = cv2.INTER_AREA) 
    plt.imshow(image, cmap=plt.cm.bone)
    plt.show()
    plt.imshow(crop, cmap=plt.cm.bone)
    plt.show()
    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    plt.show()
    plt.imshow(inter_area_resize_256, cmap=plt.cm.bone)
    plt.show()
    inter_area_resize_64 = cv2.resize(crop , (64, 64),  interpolation = cv2.INTER_AREA)
    plt.imshow(inter_area_resize_64, cmap=plt.cm.bone)
    plt.show()
    #return dataset.pixel_array


Testing visualize_single_dcm_file(path) function

In [ ]:
visualize_single_dcm_file('../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf/00ac73cfc372.dcm')

# PATHS OF SERIES IN THE TRAINING SET

In [ ]:
train_path = '../input/rsna-str-pulmonary-embolism-detection/train'
studies_in_train = os.listdir(train_path)
print(len(studies_in_train))


studies_in_train.sort()
#print(train_path+'/'+studies_in_train[0]+'/'+ os.listdir(train_path+'/'+studies_in_train[0])[0])
series_paths = []
for x in range(len(studies_in_train)):
    temp_series_path = train_path+'/'+studies_in_train[x]+'/'+ os.listdir(train_path+'/'+studies_in_train[x])[0]
    series_paths.append(temp_series_path)

print(len(series_paths))

# PREPARE Y LABELS

In [ ]:
with open ('../input/rsna-str-pulmonary-embolism-detection/train.csv') as csvfile:  
    readCSVFeatures = csv.reader(csvfile, delimiter=',')
    y = list(csv.reader(csvfile))
    
y = np.asarray(y)
study_instances = y[:,0]
unique_study_instances, indexes_study_instances = np.unique(study_instances, return_index=True)
    

In [ ]:

print(unique_study_instances[0])
unique_study_instances =np.asarray(unique_study_instances)

train_y = y[indexes_study_instances]
print(len(train_y))
train_y_cleaned = []
for i in range(0,len(studies_in_train)):
    ind = np.where(unique_study_instances==studies_in_train[i])
    train_y_cleaned.append(train_y[ind][0])
print(len(train_y_cleaned))    
train_y_cleaned= np.asarray(train_y_cleaned)
y_train = train_y_cleaned[:,3:len(train_y_cleaned[0])]


# Preprocessing of a single serie
 

* Load Scans in given folder path
* Get pixels of slices 
* Plot 3d 
Reference : https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial

In [ ]:
# Load the scans in given folder path
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices



def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    image_64 = np.ones((len(slices),64,64))
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
        cut_constant = 50
        pixel_arrays_shape = (image[slice_number]).shape
        temp = image[slice_number][:,cut_constant:pixel_arrays_shape[1]-1-cut_constant]
        temp = temp[cut_constant:pixel_arrays_shape[0]-1-cut_constant,:]
        image_64[slice_number] = cv2.resize(temp , (64, 64),  interpolation = cv2.INTER_AREA)
        #image[slice_number] = inter_area_resize_64
        
        
    

    #return np.array(image, dtype=np.int16),np.array(image_64, dtype=np.int16)
    return np.array(image_64, dtype=np.int16)




def plot_3d(image, threshold=-300):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces, norm, val = measure.marching_cubes_lewiner(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()


# TRY 
* def load_scan
* def get_pixels_hu
* plot_3d(first_patient_pixels, 400)    

In [ ]:
slices = load_scan('../input/rsna-str-pulmonary-embolism-detection/train/005a0dbcb4b7/4ceaee66edc8')
first_patient_pixels= get_pixels_hu(slices)
plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Show some slice in the middle
plt.imshow(first_patient_pixels[80], cmap=plt.cm.gray)
plt.show()

#plot_3d(first_patient_pixels, 400)    

# RESAMPLING 

In [ ]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    #return np.array(image, dtype=np.int16), np.array([slices[0].SliceThickness, slices[0].PixelSpacing[0], slices[0].PixelSpacing[1]], dtype=np.float32)
    return np.array(image, dtype=np.int16), np.array([1, 1, 1], dtype=np.float32)

#pix_resampled,spacing = resample(first_patient_pixels,slices, [1,1,1])
#plot_3d(pix_resampled, 400)


# PLOT 3D image of a given series path

In [ ]:
def plot_scans_3d(path): 
    number_of_slices_to_be_used = 300
    slices = load_scan(path)
    first_patient_pixels = get_pixels_hu(slices)
    plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
    plt.xlabel("Hounsfield Units (HU)")
    plt.ylabel("Frequency")
    plt.show()
    pix_resampled,spacing = resample(first_patient_pixels,slices, [1,1,1])
    # Show some slice in the middle
    if pix_resampled.shape[0] > number_of_slices_to_be_used:
        pix_resampled_slice_editted = pix_resampled[len(pix_resampled)-number_of_slices_to_be_used:len(pix_resampled)]
    elif pix_resampled.shape[0] < number_of_slices_to_be_used:       

        dummy_slice = -1024*np.ones((number_of_slices_to_be_used-pix_resampled.shape[0],512, 512), dtype=int)
        pix_resampled_slice_editted = np.concatenate((dummy_slice, pix_resampled), axis=0)
            
    
    
    plt.imshow(first_patient_pixels[10], cmap=plt.cm.gray)
    plt.show()
    plot_3d(first_patient_pixels, 400) 
    plot_3d(pix_resampled, 400) 
    plot_3d(pix_resampled_slice_editted, 400) 
    


In [ ]:
plot_scans_3d(series_paths[6222])

# **Normalization**


In [ ]:

MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

PIXEL_MEAN = 0.25
def zero_center(image):
    image = image - PIXEL_MEAN
    return image



# PREPROCESS CT SCANS OF A GIVEN PATH OF A SERIES

In [ ]:
def preprocess(path):
    number_of_slices_to_be_used = 50
    slices = load_scan(path)
    pixels_of_serie = get_pixels_hu(slices)
    pixels_of_serie_resampled,spacing = resample(pixels_of_serie,slices, [1,1,1])
    if pixels_of_serie_resampled.shape[0] > number_of_slices_to_be_used:
        pixels_of_serie_resampled = pixels_of_serie_resampled[len(pixels_of_serie_resampled)-number_of_slices_to_be_used:len(pixels_of_serie_resampled)]
    elif pixels_of_serie_resampled.shape[0] < number_of_slices_to_be_used:       

        dummy_slice = -1024*np.ones((number_of_slices_to_be_used-pixels_of_serie_resampled.shape[0],64, 64), dtype=int)
        pixels_of_serie_resampled = np.concatenate((dummy_slice, pixels_of_serie_resampled), axis=0)
                
    preprocessed_image = normalize(pixels_of_serie_resampled)
    preprocessed_image = zero_center(preprocessed_image)
    preprocessed_image = preprocessed_image.reshape( number_of_slices_to_be_used, 64, 64, 1)
    #preprocessed_image = preprocessed_image.reshape( 1,number_of_slices_to_be_used, 64, 64, 1)

    return preprocessed_image
    
    
    
    


# TEST 
def preprocess(path)

In [ ]:
number_of_slices_to_be_used = 50
x_train = np.ones((4,number_of_slices_to_be_used, 64, 64, 1),  dtype=np.int16)

In [ ]:
print(x_train[0].shape)
dene = preprocess(series_paths[0])
print(dene.shape)

In [ ]:


for i in range(0,4):
    try :         
        x_train[i]= preprocess(series_paths[i])
    except:
        print(i)




In [ ]:
print(y_train.shape)
print(x_train.shape)

print(x_train.shape)

In [ ]:
#delete_indexes = [5,21,25,48,99,107,115,153,185,193]

In [ ]:
#x_train  = np.delete(x_train , delete_indexes)
y_train = y_train[0:4]
#y_train = np.delete(y_train , delete_indexes)

print(y_train.shape)
print(x_train.shape)

In [ ]:
# The inputs are 28x28x28 volumes with a single channel, and the
# batch size is 4
number_of_slices_to_be_used = 50
#input_shape =( 240, 64, 64, 1)
input_layer = tf.keras.layers.Input(( number_of_slices_to_be_used, 64, 64, 1))

conv_layer1 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)

pooling_layer1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
conv_layer4 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
pooling_layer2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer2 = tf.keras.layers.BatchNormalization()(pooling_layer2)
flatten_layer = tf.keras.layers.Flatten()(pooling_layer2)

print(flatten_layer.shape)

dense_layer1 = tf.keras.layers.Dense(units=2048, activation='relu')(flatten_layer)
dense_layer1 = tf.keras.layers.Dropout(0.4)(dense_layer1)
dense_layer2 = tf.keras.layers.Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = tf.keras.layers.Dropout(0.4)(dense_layer2)
output_layer = tf.keras.layers.Dense(units=14, activation='softmax')(dense_layer2)
print(output_layer.shape)
## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:

y_train = y_train.astype(np.float)

In [ ]:
model.compile(loss=categorical_crossentropy, optimizer=Adadelta(lr=0.1), metrics=['acc'])
model.fit(x=x_train, y=y_train, batch_size=2, epochs=50, validation_split=0.2)

# CREATE 3D CONVOLUTIONAL NEURAL NETWORK